In [1]:
import pandas as pd
import pickle

import ipywidgets as widgets
from ipywidgets import interact

from src.data_load import load_tables, load_online_instance, load_distances
from src.filtering import flexible_filter
from src.plotting import plot_metrics_comparison, plot_gantt_labors_by_driver, plot_service_driver_distance, \
                         plot_gantt_by_services, plot_gantt_by_drivers
from src.metrics import collect_results_to_df, compute_metrics_with_moves, get_day_plotting_df, \
                        collect_hist_baseline_dfs
from src.experimentation_config import *
from src.config import *
from src.alpha_tuning_utils import collect_alpha_results_to_df, filter_df_on_hyperparameter_selection, include_all_city, \
    metrics, iterations_nums, alphas

data_path = '../data'

instance = 'instRD1'        # Options: ['instAS1', 'instRS1']

distance_type = 'osrm'              # Options: ['osrm', 'manhattan']
distance_method = 'haversine'      # Options: ['precalced', 'haversine']

directorio_df, labors_raw_df, cities_df, duraciones_df, valid_cities = load_tables(data_path, generate_labors=False)
labors_real_df, labors_static_df, labors_dynamic_df = load_online_instance(data_path, instance, labors_raw_df)
dist_dict = {}

fechas = fechas_dict[instance]

# Cargar resultados

In [2]:
labors_algo_offline_df, moves_algo_offline_df = collect_alpha_results_to_df(data_path, instance, distance_method, metrics, alphas, iterations_nums)

In [3]:
hist_inst = f'{instance[:5]}S{instance[6:]}'
labors_hist_df, moves_hist_df = collect_hist_baseline_dfs(data_path, hist_inst, fechas, distance_method)

hyperparameter_selection = {
    'driver_distance': {'num_iter':500, 'alpha':0},
    'driver_extra_time':    {'num_iter':500, 'alpha':0.1},
    'hybrid':               {'num_iter':500, 'alpha':0.1}
}

labors_algo_offline_df, moves_algo_offline_df = collect_alpha_results_to_df(data_path, hist_inst, distance_method, metrics, alphas, iterations_nums)
labors_algo_offline_df = filter_df_on_hyperparameter_selection(labors_algo_offline_df, hyperparameter_selection)
moves_algo_offline_df = filter_df_on_hyperparameter_selection(moves_algo_offline_df, hyperparameter_selection)

labors_hist_df = include_all_city(labors_hist_df)
moves_hist_df = include_all_city(moves_hist_df)

with open(f'{data_path}/resultados/online_operation/{instance}/res_dynamic.pkl', "rb") as f:
    labors_algo_online_df, moves_algo_online_df = pickle.load(f)  # dict: {city: (df_cleaned, df_moves, n_drivers)}


labors_algo_online_df = include_all_city(labors_algo_online_df)
moves_algo_online_df = include_all_city(moves_algo_online_df)

# Results

In [4]:
import pandas as pd
import plotly.graph_objects as go
from plotly.subplots import make_subplots

from src.utils import get_city_name_from_code
from src.alpha_tuning_utils import add_aggregated_totals

def plot_metrics_comparison_online_offline(
    labors_hist_df: pd.DataFrame,
    moves_hist_df: pd.DataFrame,
    labors_algo_offline_df: pd.DataFrame,
    moves_algo_offline_df: pd.DataFrame,
    labors_algo_online_df: pd.DataFrame,
    moves_algo_online_df: pd.DataFrame,
    city: str,
    metricas,
    dist_dict: dict,
    fechas: tuple[str, str], 
    group_by=None,
    xaxis_mode: str = "date"
):
    """
    Compares historic (real), offline algorithm, and online algorithm metrics.
    Shows two metrics per figure (side-by-side), with clean legends and colors.

    Parameters
    ----------
    city : str
        City code or 'ALL'.
    xaxis_mode : {"date", "day"}
        "date" -> show actual dates on X axis
        "day" -> show 'day1', 'day2', ...
    """

    # --- Compute metrics for each source ---
    metrics_real_df = compute_metrics_with_moves(
        labors_hist_df, moves_hist_df, fechas, dist_dict,
        workday_hours=8, city=city, assignment_type='historic',
        skip_weekends=False, dist_method='haversine'
    )
    metrics_offline_df = compute_metrics_with_moves(
        labors_algo_offline_df, moves_algo_offline_df, fechas, dist_dict,
        workday_hours=8, city=city, assignment_type='algorithm',
        skip_weekends=False, dist_method='haversine'
    )
    metrics_online_df = compute_metrics_with_moves(
        labors_algo_online_df, moves_algo_online_df, fechas, dist_dict,
        workday_hours=8, city=city, assignment_type='algorithm',
        skip_weekends=False, dist_method='haversine'
    )

    # --- Grouping (if needed) ---
    if group_by is not None:
        metrics_real_df = add_aggregated_totals(metrics_real_df, group_by=group_by)
        metrics_offline_df = add_aggregated_totals(metrics_offline_df, group_by=group_by)
        metrics_online_df = add_aggregated_totals(metrics_online_df, group_by=group_by)

    # --- Label mapping ---
    labels = {
        "vt_count": "Labores tipo Vehicle Transportation",
        "num_drivers": "Número de conductores",
        "labores_por_conductor": "Labores por conductor",
        "utilizacion_promedio_%": "Utilización promedio (%)",
        "total_distance": "Distancia total (km)",
        "driver_move_distance": "Distancia recorrida por conductores (km)",
        "labor_extra_time": "Tiempo extra total por labor (min)",
        "driver_extra_time": "Tiempo extra total por conductor (min)",
    }

    if isinstance(metricas, str):
        metricas = [metricas]

    # --- X-axis setup ---
    x_real = pd.to_datetime(metrics_real_df["day"])
    x_off = pd.to_datetime(metrics_offline_df["day"])
    x_on = pd.to_datetime(metrics_online_df["day"])

    if xaxis_mode == "day":
        day_labels = [f"day{i+1}" for i in range(len(x_real))]
        x_real_display = x_off_display = x_on_display = day_labels
    else:
        x_real_display, x_off_display, x_on_display = x_real, x_off, x_on

    # 🎨 Color palette & styles
    color_real = "#800080"     # purple – historic
    color_off  = "#17BECF"     # blue – offline algorithm
    color_on   = "#2ECC71"     # sexy green – online algorithm

    figs = []
    for i in range(0, len(metricas), 2):
        pair = metricas[i:i+2]
        fig = make_subplots(
            rows=1, cols=len(pair),
            subplot_titles=[labels.get(m, m) for m in pair]
        )

        for j, metrica in enumerate(pair, start=1):
            label = labels.get(metrica, metrica)

            # Historic
            fig.add_trace(go.Scatter(
                x=x_real_display, y=metrics_real_df[metrica],
                mode="lines+markers",
                name=f"{label} (Real)",
                line=dict(color=color_real, dash="solid", width=2),
                marker=dict(symbol="circle", size=7, color=color_real),
                legendgroup=f"{metrica}_real",
                showlegend=True
            ), row=1, col=j)

            # Offline algorithm
            fig.add_trace(go.Scatter(
                x=x_off_display, y=metrics_offline_df[metrica],
                mode="lines+markers",
                name=f"{label} (Offline)",
                line=dict(color=color_off, dash="dash", width=2),
                marker=dict(symbol="x", size=7, color=color_off),
                legendgroup=f"{metrica}_offline",
                showlegend=True
            ), row=1, col=j)

            # Online algorithm (new 🌿)
            fig.add_trace(go.Scatter(
                x=x_on_display, y=metrics_online_df[metrica],
                mode="lines+markers",
                name=f"{label} (Online)",
                line=dict(color=color_on, dash="dot", width=2),
                marker=dict(symbol="triangle-up", size=8, color=color_on),
                legendgroup=f"{metrica}_online",
                showlegend=True
            ), row=1, col=j)

            # Axis formatting
            fig.update_xaxes(title="Día", row=1, col=j)
            fig.update_yaxes(title=label, row=1, col=j)

        # --- Layout polish ---
        fig.update_layout(
            height=500,
            width=1000,
            title=(
                f"Métricas — {get_city_name_from_code(city)} "
                f"({fechas[0]} a {fechas[1]})"
            ),
            margin=dict(t=100, b=100),
            legend=dict(
                orientation="h",
                yanchor="top", y=-0.25,
                xanchor="center", x=0.5,
                font=dict(size=11)
            ),
            plot_bgcolor="white"
        )

        fig.update_xaxes(showgrid=True, gridwidth=0.5, gridcolor="lightgray")
        fig.update_yaxes(showgrid=True, gridwidth=0.5, gridcolor="lightgray")

        fig.show()
        figs.append(fig)

    return figs


In [5]:
import ipywidgets as widgets
from ipywidgets import interact

metricas = ['vt_count', 'num_drivers', "labores_por_conductor", 'total_distance', 
            'labor_extra_time', 'driver_extra_time', 'driver_move_distance']

def plot_metrics_comparison_interactive_online_offline(
    labors_hist_df: pd.DataFrame,
    moves_hist_df: pd.DataFrame,
    labors_algo_offline_df: pd.DataFrame,
    moves_algo_offline_df: pd.DataFrame,
    labors_algo_online_df: pd.DataFrame,
    moves_algo_online_df: pd.DataFrame,
    cities: list[str],
    metrics: list[str], 
    metricas,
    dist_dict: dict,
    fechas: tuple[str, str]
):
    """
    Interactive wrapper for comparing:
        - Historic baseline
        - Offline algorithm
        - Online algorithm (dynamic)

    Allows selecting city & metric interactively with ipywidgets.
    """

    def _plot_for_city(city: str, metric: str):
        """Inner function that updates plots when dropdown changes."""

        # Parameters used for filtering offline algorithm results
        alphas_selection = {
            'hybrid': 0.1,
            'driver_extra_time': 0.1,
            'driver_distance': 0
        }
        alpha = alphas_selection.get(metric, 0.1)
        num_iter = 500

        # Filter offline algorithm subset
        labors_off_temp = labors_algo_offline_df[
            (labors_algo_offline_df['metric'] == metric) &
            (labors_algo_offline_df['alpha'] == alpha) &
            (labors_algo_offline_df['num_iter'] == num_iter)
        ].copy()
        moves_off_temp = moves_algo_offline_df[
            (moves_algo_offline_df['metric'] == metric) &
            (moves_algo_offline_df['alpha'] == alpha) &
            (moves_algo_offline_df['num_iter'] == num_iter)
        ].copy()

        # Ensure 'ALL' aggregate city is included
        for df_temp in [labors_off_temp, moves_off_temp,
                        labors_algo_online_df, moves_algo_online_df,
                        labors_hist_df, moves_hist_df]:
            if "city" in df_temp.columns:
                if "ALL" not in df_temp["city"].unique():
                    all_city_df = df_temp.copy()
                    all_city_df["city"] = "ALL"
                    df_temp = pd.concat([df_temp, all_city_df])
        
        # ✅ Call updated comparison function
        figs = plot_metrics_comparison_online_offline(
            labors_hist_df=labors_hist_df,
            moves_hist_df=moves_hist_df,
            labors_algo_offline_df=labors_off_temp,
            moves_algo_offline_df=moves_off_temp,
            labors_algo_online_df=labors_algo_online_df,
            moves_algo_online_df=moves_algo_online_df,
            city=city,
            metricas=metricas,
            dist_dict=dist_dict,
            fechas=fechas
        )
        # return figs

    # --- Interactive dropdown widgets ---
    city_dropdown = widgets.Dropdown(
        options=['ALL'] + sorted(list(set(cities))),
        value=cities[0] if cities else 'ALL',
        description="Ciudad:",
        style={"description_width": "initial"},
        layout=widgets.Layout(width="300px")
    )

    metric_dropdown = widgets.Dropdown(
        options=metrics,
        value=metrics[0],
        description="Métrica:",
        style={"description_width": "initial"},
        layout=widgets.Layout(width="300px")
    )

    # --- Combine interactive controls ---
    interact(
        _plot_for_city,
        city=city_dropdown,
        metric=metric_dropdown
    )


plot_metrics_comparison_interactive_online_offline(
    labors_hist_df,
    moves_hist_df,
    labors_algo_offline_df,
    moves_algo_offline_df,
    labors_algo_online_df,
    moves_algo_online_df,
    cities=valid_cities,   # list of cities
    metrics=metrics,       # e.g. ['hybrid', 'driver_extra_time', 'driver_distance']
    metricas=metricas,     # list of metric names you want plotted
    dist_dict=dist_dict,
    fechas=(fechas[0], fechas[-1])
)


interactive(children=(Dropdown(description='Ciudad:', index=4, layout=Layout(width='300px'), options=('ALL', '…

In [6]:
moves_hist_df

,service_id,labor_id,labor_context_id,labor_name,labor_category,historic_driver,schedule_date,historic_start,historic_end,start_point,end_point,distance_km,duration_min,city,date
150,268652,365656,365656_free,FREE_TIME,FREE_TIME,5455,2025-07-21 07:30:00-05:00,2025-07-21 06:42:58.308825-05:00,2025-07-21 06:42:58.308825-05:00,POINT (-75.5568613 6.278588999999999),POINT (-75.5568613 6.278588999999999),0.000000,0.0,1,2025-07-21
151,268652,365656,365656_move,DRIVER_MOVE,DRIVER_MOVE,5455,2025-07-21 07:30:00-05:00,2025-07-21 06:42:58.308825-05:00,2025-07-21 07:00:00-05:00,POINT (-75.5568613 6.278588999999999),POINT (-75.56948779999999 6.203055700000001),8.514093,17.0,1,2025-07-21
152,268652,365656,365656_labor,Alfred Initial Transport,VEHICLE_TRANSPORTATION,5455,2025-07-21 07:30:00-05:00,2025-07-21 07:00:00-05:00,2025-07-21 07:49:05.143231-05:00,POINT (-75.56948779999999 6.203055700000001),POINT (-75.5697983 6.227549600000001),NaN,49.1,1,2025-07-21
156,269410,366473,366473_free,FREE_TIME,FREE_TIME,31857,2025-07-21 08:00:00-05:00,2025-07-21 07:22:09.047568-05:00,2025-07-21 07:22:09.047568-05:00,POINT (-75.58290559999999 6.155980599999999),POINT (-75.58290559999999 6.155980599999999),0.000000,0.0,1,2025-07-21
157,269410,366473,366473_move,DRIVER_MOVE,DRIVER_MOVE,31857,2025-07-21 08:00:00-05:00,2025-07-21 07:22:09.047568-05:00,2025-07-21 07:30:00-05:00,POINT (-75.58290559999999 6.155980599999999),POINT (-75.616297 6.144),3.924604,7.8,1,2025-07-21
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1465,274014,371416,371416_move,DRIVER_MOVE,DRIVER_MOVE,6412,2025-07-25 16:30:00-05:00,2025-07-25 15:58:10.098320-05:00,2025-07-25 16:00:00-05:00,POINT (-73.1160807 7.116271499999999),POINT (-73.1134749 7.1084515),0.915847,1.8,ALL,2025-07-25
1466,274014,371416,371416_labor,Alfred Initial Transport,VEHICLE_TRANSPORTATION,6412,2025-07-25 16:30:00-05:00,2025-07-25 16:00:00-05:00,2025-07-25 16:57:47.459696-05:00,POINT (-73.1134749 7.1084515),POINT (-73.1246665 7.0325719),NaN,57.8,ALL,2025-07-25
1605,274328,371745,371745_free,FREE_TIME,FREE_TIME,6412,2025-07-26 11:30:00-05:00,2025-07-26 09:00:00-05:00,2025-07-26 10:55:44.773032-05:00,POINT (-73.1037471 7.0910492),POINT (-73.1037471 7.0910492),0.000000,115.7,ALL,2025-07-26
1606,274328,371745,371745_move,DRIVER_MOVE,DRIVER_MOVE,6412,2025-07-26 11:30:00-05:00,2025-07-26 10:55:44.773032-05:00,2025-07-26 11:00:00-05:00,POINT (-73.1037471 7.0910492),POINT (-73.11345413684005 7.107574257887976),2.126891,4.3,ALL,2025-07-26
